In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
import pandas as pd
from owlready2 import get_ontology
from elasticsearch import Elasticsearch
from mapping import *

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
config = config_read('../')

# Load owl file
data_path = config['owl']['path']
onto = get_ontology(data_path).load()

# Elasticsearch
server_ip = config['elasticsearch']['ip']
index_name = config['elasticsearch']['name']
es = Elasticsearch(server_ip)

In [4]:
unit_path_df = pd.read_csv('../unit_path_for_mapping.csv')

In [48]:
query_terms = ['용감한 형제', '작곡', '노래', '장르']
# query_terms = ['소녀시대', '부른', '노래']
# query_terms = ['지드래곤', '구성원', '그룹']
# query_terms = ['빅뱅', '리더', '본명']

query_patitions = query_patitioning(query_terms)
p_star, score_list = partition_scoring(es, index_name, query_patitions)
candidates_list = [onto.search(label=query) for query in p_star]
resource_combinations = list(itertools.product(*candidates_list))
resource_combinations

[(data.용감한_형제, data.isComposedBy, mo.Track, data.hasGenre),
 (data.용감한_형제, data.isComposedBy, mo.Track, data.Genre)]

In [50]:
conceptual_graph = []

for rc in resource_combinations:
    conceptual_arc = []
    state_list = []
    rc = list(rc)

    for r in rc:
        if r in onto.object_properties():
            state_list.append('edge') 
        elif r in onto.data_properties():
            state_list.append('edge') 
        else:
            state_list.append('vertex') 

    # 마지막이 edge일 때
    if state_list[-1] == 'edge':
        state_list.append('vertex')
        rc.append('owl.Thing')

    # vertex - vertex 일때
    index_list = []
    for i in range(len(state_list)-1):
        if state_list[i] == 'vertex' and state_list[i+1] == 'vertex':
            index_list.append(i+1)
    
    for i in index_list:
        state_list.insert(i, 'edge')
        rc.insert(i, 'Any P')
    
    for offset in range(0,len(rc)-2, 2):
        conceptual_arc.append(tuple(rc[offset:offset+3]))
    
    print(state_list)
    print(conceptual_arc)

    conceptual_graph.append(conceptual_arc)

# for conceptual_arc in conceptual_graph:
#     print(conceptual_arc)

['vertex', 'edge', 'vertex', 'edge', 'vertex']
[(data.용감한_형제, data.isComposedBy, mo.Track), (mo.Track, data.hasGenre, 'owl.Thing')]
['vertex', 'edge', 'vertex', 'edge', 'vertex']
[(data.용감한_형제, data.isComposedBy, mo.Track), (mo.Track, 'Any P', data.Genre)]


In [46]:
conceptual_arc = {}
df_list = []

for conceptual_arc in conceptual_graph:
    for s, p, o in conceptual_arc:
        # print(unit_path_df[unit_path_df['S'] == s and unit_path_df['P'] == p and unit_path_df['O'] == o])
        s_cond = unit_path_df['S'] == str(s)
        p_cond = unit_path_df['P'] == str(p)
        # o_cond = unit_path_df['range'] == str(o)

        df = unit_path_df.loc[s_cond & p_cond, ['S', 'P', 'O', 'W']]
    
    df_list.append(df)

In [47]:
pd.concat(df_list)

,S,P,O,W
1001,data.빅뱅,data.hasLeader,data.지드래곤,0.75
